In [1]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled,function_tool
from agents.extensions.models.litellm_model import LitellmModel

from dotenv import load_dotenv
import os
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

BASE_URL = os.getenv("EXAMPLE_BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/"
API_KEY = os.getenv("EXAMPLE_API_KEY") or gemini_api_key
MODEL_NAME = os.getenv("EXAMPLE_MODEL_NAME") or "gemini-2.0-flash"
# print(API_KEY)
import nest_asyncio #this is needed to run async code in Jupyter notebooks
nest_asyncio.apply()
client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)
set_tracing_disabled(disabled=True)

## this is main agent will handoff work to other related agents

## subagent

In [ ]:
@function_tool
def turn_right(degrees: int) -> str:
    """turn right or right..."""
    return f"Turning right by {degrees} "
@function_tool
def turn_left(degrees: int) -> str:
    return f"Turning left by {degrees} degrees."
@function_tool
def move_forward(distance: int) -> str:
    return f"Moving forward by {distance} units."
@function_tool
def move_backward(distance: int) -> str:
    return f"Moving backward by {distance} units."

# subagent1
wheel_control_agent = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME,openai_client=client),
    name="wheel_control_agent",
    instructions="you are an expert in controlling the wheels of a robot. You can move the robot forward, backward, turn left, and turn right. You understand commands related to movement and can",
    tools=[turn_right, turn_left, move_forward, move_backward],
    handoff_description="Use this agent to control the robot's wheels based on movement commands.")

In [13]:
@function_tool
def fan_on(temp : str) -> str:
    if temp > "75":
        return "Turning the fan on."
    else:
        return "The temperature is comfortable, no need to turn on the fan."
@function_tool
def fan_off(temp : str) -> str:
    if temp < "70":
        return "Turning the fan off."
    else:
        return "The temperature is still high, no need to turn off the fan."

# subagent2
enviroment_control_agent = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME,openai_client=client),
    name="environment_control_agent",
    instructions="you are an expert in controlling the environment of a robot. You can adjust temperature ",
    tools=[fan_on, fan_off],
    handoff_description="Use this agent to control the robot's environment based on environmental commands.")

In [14]:
mainAgetn = Agent(
    model=OpenAIChatCompletionsModel(model=MODEL_NAME,openai_client=client),
    name="supervisor",
    instructions="you are main agent ",
    handoffs=[wheel_control_agent,enviroment_control_agent]
)

In [ ]:
result = await Runner.run(
    mainAgetn,
    "turn the robot 20 degree right ",
)

In [16]:
result.final_output

'OK. I have turned the robot 20 degrees to the right.\n'

In [17]:
print(dir(result)) 

['__abstractmethods__', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_last_agent', 'context_wrapper', 'final_output', 'final_output_as', 'input', 'input_guardrail_results', 'last_agent', 'last_response_id', 'new_items', 'output_guardrail_results', 'raw_responses', 'to_input_list']
